In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
from PIL import Image
import torchvision.models as models
import copy
from torchvision.utils import save_image
import PIL

In [2]:
aug_transform = transforms.Compose([torchvision.transforms.RandomAffine(degrees = 4.5, scale = (0.9, 1.1), shear = 4),
                                    torchvision.transforms.RandomHorizontalFlip(0.5),
                                    transforms.ToTensor()])

vgg19_bn = models.vgg19_bn(pretrained=True)

def getLabelAndIndexFromFileName(fn):
    _fn = fn.split('/').pop()
    _list = _fn.split("_")
    label = int(_list[0])
    index = int(_list[1].split(".")[0])
    return label, index

class imgData(Dataset):
    def __init__(self, root, transform = None):
        self.filenames = []
        self.root = root
        self.transform = transform
        filenames = glob.glob(os.path.join(self.root, '*.png'))
        for fn in filenames:
            label, index = getLabelAndIndexFromFileName(fn)
            self.filenames.append((fn, label)) # (filename, label) pair
        self.len = len(self.filenames) 
    
    def __getitem__(self, index):
        with torch.no_grad():
            fn, label = self.filenames[index]
            image = Image.open(fn)
            if self.transform is not None:
                image = self.transform(image)
            return image, label
    
    def __len__(self):
        return self.len

In [3]:
trainset = imgData(root='hw2_data/p1_data/train_50', transform=aug_transform)
testset = imgData(root='hw2_data/p1_data/val_50', transform=transforms.ToTensor())

print('# images in trainset:', len(trainset))
print('# images in testset:', len(testset))

# Use the torch dataloader to iterate through the dataset
trainset_loader = DataLoader(trainset, batch_size=2048, shuffle=True, num_workers=0)
testset_loader = DataLoader(testset, batch_size=2500, shuffle=False, num_workers=0)

# get some random training images
dataiter = iter(trainset_loader)
images, labels = dataiter.next()

print('Image tensor in each batch:', images.shape, images.dtype)
print('Label tensor in each batch:', labels.shape, labels.dtype)

# images in trainset: 22500
# images in testset: 2500
Image tensor in each batch: torch.Size([2048, 3, 32, 32]) torch.float32
Label tensor in each batch: torch.Size([2048]) torch.int64


In [4]:
# activate cuda
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda:0" if use_cuda else "cpu")
#device = torch.device('cpu')
print('Device used:', device)

Device used: cuda:0


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            vgg19_bn
        )
        self.fc1 = nn.Sequential(
            nn.Linear(1000, 50)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.fc1(x)
        return x

In [6]:
def train(model, epoch, log_interval=2):
    optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 1e-3)
    criterion = nn.CrossEntropyLoss()
    model.train()
    iteration = 0
    for ep in range(epoch):
        print('')
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            optimizer.zero_grad()
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1
        test(model) # Evaluate at the end of each epoch

def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)       
        
currentBestPercentage = 0  
def test(model):
    criterion = nn.CrossEntropyLoss()
    model.eval()  # Important: set evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad(): # This will free the GPU memory used for back-prop
        for data, target in testset_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()*len(data) # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(testset_loader.dataset)
    percentage = 100. * correct / len(testset_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(testset_loader.dataset), percentage))
          
    global currentBestPercentage
    if currentBestPercentage < percentage and 60.0 < percentage:
        print('current best model updated!')
        currentBestPercentage = percentage
        optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 1e-3)
        save_checkpoint('p1_current_best-'+str(round(percentage,4))+'.pth', model, optimizer)

In [7]:
model = Net()
model.to(device)
print(model)

Net(
  (features): Sequential(
    (0): VGG(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (9): ReLU(inplace=True)
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (12): ReLU(inplace=True)
        (13): 

In [ ]:
train(model, 200)


Train Epoch: 0 [0/22500 (0%)]	Loss: 5.141318
Train Epoch: 0 [4096/22500 (18%)]	Loss: 4.822347
Train Epoch: 0 [8192/22500 (36%)]	Loss: 4.536621
Train Epoch: 0 [12288/22500 (55%)]	Loss: 4.226572
Train Epoch: 0 [16384/22500 (73%)]	Loss: 3.989297
Train Epoch: 0 [20200/22500 (91%)]	Loss: 3.902361

Test set: Average loss: 4.1519, Accuracy: 123/2500 (5%)


Train Epoch: 1 [2048/22500 (9%)]	Loss: 4.090623
Train Epoch: 1 [6144/22500 (27%)]	Loss: 3.895405
Train Epoch: 1 [10240/22500 (45%)]	Loss: 3.776376
Train Epoch: 1 [14336/22500 (64%)]	Loss: 3.657785
Train Epoch: 1 [18432/22500 (82%)]	Loss: 3.637990

Test set: Average loss: 3.4649, Accuracy: 288/2500 (12%)


Train Epoch: 2 [0/22500 (0%)]	Loss: 3.500386
Train Epoch: 2 [4096/22500 (18%)]	Loss: 3.479666
Train Epoch: 2 [8192/22500 (36%)]	Loss: 3.348505
Train Epoch: 2 [12288/22500 (55%)]	Loss: 3.275608
Train Epoch: 2 [16384/22500 (73%)]	Loss: 3.159861
Train Epoch: 2 [20200/22500 (91%)]	Loss: 2.991932

Test set: Average loss: 2.9171, Accuracy: 557/